$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}$

# Assignment 6: Min-Conflicts

*Namita Kharat*

For this assignment, I have used the `min-conflicts` code from the lecture notes to solve the following scheduling problem. 

I am supposed to assign classes to classrooms and times for the department. The scheduling is simplified by the fact at this imaginary university each class meets every day. 

I need to schedule only thee class rooms:

  * CSB 130
  * CSB 325
  * CSB 425
  
Classes start on the hour. I can only assign classes to the hours of

  * 9 am
  * 10 am
  * 11 am
  * 12 pm
  *  1 pm
  *  2 pm
  *  3 pm
  *  4 pm
  
These 22 classes must be scheduled:

  * CS160, CS163, CS164,
  * CS220, CS270, CS253,
  * CS320, CS314, CS356, CS370,
  * CS410, CS414, CS420, CS430, CS440, CS445, CS453, CS464,
  * CS510, CS514, CS535, CS540, CS545

The schedule must not violate any of the following constraints.

  * Two classes cannot meet in the same room at the same time.
  * Classes with the same first digit cannot meet at the same time, because students might take a subset of these in one semester. There is one exception to this rule. CS163 and CS164 can meet at the same time.

The variables for this CSP problem are the classes.  The values assigned to each class will be a tuple containing a room and a time.

## Required Functions

     assignments, steps = schedule(classes, times, rooms, max_steps)
     
     # classes: list of all class names, like 'CS410'
     # times: list of all start times, like '10 am' and ' 1 pm'
     # rooms: list of all rooms, like 'CSB 325'
     # max_steps: maximum number of assignments to try
     # assignments: dictionary of values assigned to variables, like {'CS410': ('CSB 425', '10 am'), ...}
     # steps: actual number of assignments tested before solution found
     #    assignments and steps will each be None if solution was not found.
     
     result = constraints_ok(class_name_1, value_1, class_name_2, value_2)
     
     # class_name_1: as above, like 'CS410'
     # value_1: tuple containing room and time.
     # class_name_2: a second class name
     # value_2: another tuple containing a room and time
     # result: True of the assignment of value_1 to class_name 1 and value_2 to class_name 2 does
     #         not violate any constraints.  False otherwise.
     
     display(assignments, rooms, times)
     
     # assignments: returned from call to your schedule function
     # rooms: list of all rooms as above
     # times: list of all times as above
     

## Description:

**constraints_ok(class_name_1, value_1, class_name_2, value_2):** This function checks whether its arguments violates any constraints. Two class names are passed and their corresponding times and rooms are passed in a tuple.

**schedule(classes, times, rooms, max_steps):** It's a wrapper function used to call the min_conflicts function. Parameters like domains and neighbors required by min_conflicts is also defined in this function. Domains is dictionary of classes as keys and all possible combinations of times and rooms for that class as its value. Neighbors is another dictionary consisting of classes as keys and values as classes other than the key itself.

**display(assignments, rooms, times):** This function is used to display the schedule in a readable format.

In [1]:
import random

def min_conflicts(vars, domains, constraints, neighbors, max_steps=1000): 
    """Solve a CSP by stochastic hillclimbing on the number of conflicts."""
    # Generate a complete assignment for all vars (probably with conflicts)
    current = {}
    for var in vars:
        val = min_conflicts_value(var, current, domains, constraints, neighbors)
        current[var] = val
    # Now repeatedly choose a random conflicted variable and change it
    for i in range(max_steps):
        conflicted = conflicted_vars(current,vars,constraints,neighbors)
        if not conflicted:
            return (current,i)
        var = random.choice(conflicted)
        val = min_conflicts_value(var, current, domains, constraints, neighbors)
        current[var] = val
    return (None,None)

def min_conflicts_value(var, current, domains, constraints, neighbors):
    """Return the value that will give var the least number of conflicts.
    If there is a tie, choose at random."""
    return argmin_random_tie(domains[var],
                             lambda val: nconflicts(var, val, current, constraints, neighbors)) 

def conflicted_vars(current,vars,constraints,neighbors):
    "Return a list of variables in current assignment that are in conflict"
    return [var for var in vars
            if nconflicts(var, current[var], current, constraints, neighbors) > 0]

def nconflicts(var, val, assignment, constraints, neighbors):
    "Return the number of conflicts var=val has with other variables."
    # Subclasses may implement this more efficiently
#     print(var, val, var2, val2)
#     print(constraints(var, val, var2, val2))
    def conflict(var2):
        val2 = assignment.get(var2, None)
#         print(var, val)
#         res=constraints(var, val, var2, val2)
#         print(res)
        return val2 != None and not constraints(var, val, var2, val2)
    return len(list(filter(conflict, neighbors[var])))

def argmin_random_tie(seq, fn):
    """Return an element with lowest fn(seq[i]) score; break ties at random.
    Thus, for all s,f: argmin_random_tie(s, f) in argmin_list(s, f)"""
    best_score = fn(seq[0]); n = 0
    for x in seq:
        x_score = fn(x)
        if x_score < best_score:
            best, best_score = x, x_score; n = 1
        elif x_score == best_score:
            n += 1
            if random.randrange(n) == 0:
                    best = x
    return best

In [3]:
def constraints_ok(class_name_1, value_1, class_name_2, value_2):
    # Check for classes not with same digit or the two classes are 'CS163' and 'CS164' 
    if (((class_name_1[2] != class_name_2[2])) or ((class_name_1 == 'CS163') and (class_name_2 == 'CS164')) or ((class_name_1 == 'CS164') and (class_name_2 == 'CS163'))):
        # Return True if rooms are different and times are same or vice versa or if both time and room is different for both the classes
        if (((value_1[0] != value_2[0]) and (value_1[1] == value_2[1])) or ((value_1[0] == value_2[0]) and (value_1[1] != value_2[1])) or ((value_1[0] != value_2[0]) and (value_1[1] != value_2[1]))):
            return True
        else:
            return False
    # Check for classes with same digit or the two classes are not 'CS163' and 'CS164' 
    if ((class_name_1[2] == class_name_2[2]) and ((class_name_1 != 'CS163' and class_name_2 != 'CS164') or (class_name_1 != 'CS164' and class_name_2 != 'CS163'))):
        # Return True if times are different for both the classes
        if (((value_1[0] != value_2[0]) and (value_1[1] != value_2[1])) or ((value_1[0] == value_2[0]) and (value_1[1] != value_2[1]))):
            return True
        else:
            return False
        
def schedule(classes, times, rooms, max_steps):
    rooms_times = [(x, y) for x in rooms for y in times]
    domains={key: rooms_times for key in classes}
    neighbors = {var: [v for v in classes if v != var] for var in classes}
    # Call the min_conflicts function
    assignment, steps = min_conflicts(classes,domains,constraints_ok,neighbors)
    return assignment, steps   

def display(assignments, rooms, times):
    # Initialize Lists
    NineList=[]
    TenList=[]
    ElevenList=[]
    TwelveList=[]
    OneList=[]
    TwoList=[]
    ThreeList=[]
    FourList=[]
    
    # Append the list with rooms and classes
    for i in assignments:
        if assignments[i][1] == ' 9 am':
            NineList.append((assignments[i][0],i))
    for i in assignments:
        if assignments[i][1] == '10 am':
            TenList.append((assignments[i][0],i))
    for i in assignments:
        if assignments[i][1] == '11 am':
            ElevenList.append((assignments[i][0],i))
    for i in assignments:
        if assignments[i][1] == '12 pm':
            TwelveList.append((assignments[i][0],i))
    for i in assignments:
        if assignments[i][1] == ' 1 pm':
            OneList.append((assignments[i][0],i))
    for i in assignments:
        if assignments[i][1] == ' 2 pm':
            TwoList.append((assignments[i][0],i))
    for i in assignments:
        if assignments[i][1] == ' 3 pm':
            ThreeList.append((assignments[i][0],i))
    for i in assignments:
        if assignments[i][1] == ' 4 pm':
            FourList.append((assignments[i][0],i))
    
    # Append the list with rooms having no classes
    if not any('CSB 130' in x for x in NineList):
        NineList.append((('CSB 130','     ')))
    if not any('CSB 325' in x for x in NineList):
        NineList.append((('CSB 325','     ')))
    if not any('CSB 425' in x for x in NineList):
        NineList.append((('CSB 425','     ')))

    if not any('CSB 130' in x for x in TenList):
        TenList.append((('CSB 130','     ')))
    if not any('CSB 325' in x for x in TenList):
        TenList.append((('CSB 325','     ')))
    if not any('CSB 425' in x for x in TenList):
        TenList.append((('CSB 425','     ')))
    
    if not any('CSB 130' in x for x in ElevenList):
        ElevenList.append((('CSB 130','     ')))
    if not any('CSB 325' in x for x in ElevenList):
        ElevenList.append((('CSB 325','     ')))
    if not any('CSB 425' in x for x in ElevenList):
        ElevenList.append((('CSB 425','     ')))

    if not any('CSB 130' in x for x in TwelveList):
        TwelveList.append((('CSB 130','     ')))
    if not any('CSB 325' in x for x in TwelveList):
        TwelveList.append((('CSB 325','     ')))
    if not any('CSB 425' in x for x in TwelveList):
        TwelveList.append((('CSB 425','     ')))
    
    if not any('CSB 130' in x for x in OneList):
        OneList.append((('CSB 130','     ')))
    if not any('CSB 325' in x for x in OneList):
        OneList.append((('CSB 325','     ')))
    if not any('CSB 425' in x for x in OneList):
        OneList.append((('CSB 425','     ')))

    if not any('CSB 130' in x for x in TwoList):
        TwoList.append((('CSB 130','     ')))
    if not any('CSB 325' in x for x in TwoList):
        TwoList.append((('CSB 325','     ')))
    if not any('CSB 425' in x for x in TwoList):
        TwoList.append((('CSB 425','     ')))
    
    if not any('CSB 130' in x for x in ThreeList):
        ThreeList.append((('CSB 130','     ')))
    if not any('CSB 325' in x for x in ThreeList):
        ThreeList.append((('CSB 325','     ')))
    if not any('CSB 425' in x for x in ThreeList):
        ThreeList.append((('CSB 425','     ')))

    if not any('CSB 130' in x for x in FourList):
        FourList.append((('CSB 130','     ')))
    if not any('CSB 325' in x for x in FourList):
        FourList.append((('CSB 325','     ')))
    if not any('CSB 425' in x for x in FourList):
        FourList.append((('CSB 425','     ')))

    # Sort the list according to the rooms    
    NineAMClasses=sorted(NineList)
    TenAMClasses=sorted(TenList)
    ElevenAMClasses=sorted(ElevenList)
    TwelvePMClasses=sorted(TwelveList)
    OnePMClasses=sorted(OneList)
    TwoPMClasses=sorted(TwoList)
    ThreePMClasses=sorted(ThreeList)
    FourPMClasses=sorted(FourList)
    
    # Append the list with time
    NineAMClasses = [NineAMClasses[1] for NineAMClasses in NineAMClasses]
    NineAMClasses=[' 9 am']+NineAMClasses
    TenAMClasses = [TenAMClasses[1] for TenAMClasses in TenAMClasses]
    TenAMClasses=['10 am']+TenAMClasses
    ElevenAMClasses = [ElevenAMClasses[1] for ElevenAMClasses in ElevenAMClasses]
    ElevenAMClasses=['11 am']+ElevenAMClasses
    TwelvePMClasses = [TwelvePMClasses[1] for TwelvePMClasses in TwelvePMClasses]
    TwelvePMClasses=['12 pm']+TwelvePMClasses
    OnePMClasses = [OnePMClasses[1] for OnePMClasses in OnePMClasses]
    OnePMClasses=[' 1 pm']+OnePMClasses
    TwoPMClasses = [TwoPMClasses[1] for TwoPMClasses in TwoPMClasses]
    TwoPMClasses=[' 2 pm']+TwoPMClasses
    ThreePMClasses = [ThreePMClasses[1] for ThreePMClasses in ThreePMClasses]
    ThreePMClasses=[' 3 pm']+ThreePMClasses
    FourPMClasses = [FourPMClasses[1] for FourPMClasses in FourPMClasses]
    FourPMClasses=[' 4 pm']+FourPMClasses
    
    #Print in a readable format
    print('         ','CSB 130','  ','CSB 325','  ', 'CSB 425')
    print('----------------------------------------')
    print('      '.join(NineAMClasses))
    print('      '.join(TenAMClasses))
    print('      '.join(ElevenAMClasses))
    print('      '.join(TwelvePMClasses))
    print('      '.join(OnePMClasses))
    print('      '.join(TwoPMClasses))
    print('      '.join(ThreePMClasses))
    print('      '.join(FourPMClasses))

## Examples

In [4]:
classes = ['CS160', 'CS163', 'CS164',
           'CS220', 'CS270', 'CS253',
           'CS320', 'CS314', 'CS356', 'CS370',
           'CS410', 'CS414', 'CS420', 'CS430', 'CS440', 'CS445', 'CS453', 'CS464',
           'CS510', 'CS514', 'CS535', 'CS540', 'CS545']

times = [' 9 am',
         '10 am',
         '11 am',
         '12 pm',
         ' 1 pm',
         ' 2 pm',
         ' 3 pm',
         ' 4 pm']

rooms = ['CSB 130', 'CSB 325', 'CSB 425']

In [5]:
max_steps = 100
assignments, steps = schedule(classes, times, rooms, max_steps)
print('Took', steps, 'steps')
print(assignments)

Took 2 steps
{'CS160': ('CSB 425', ' 2 pm'), 'CS163': ('CSB 130', ' 3 pm'), 'CS164': ('CSB 325', '11 am'), 'CS220': ('CSB 325', '12 pm'), 'CS270': ('CSB 425', ' 9 am'), 'CS253': ('CSB 325', ' 4 pm'), 'CS320': ('CSB 325', ' 2 pm'), 'CS314': ('CSB 425', ' 1 pm'), 'CS356': ('CSB 130', '10 am'), 'CS370': ('CSB 130', ' 4 pm'), 'CS410': ('CSB 325', ' 9 am'), 'CS414': ('CSB 130', '12 pm'), 'CS420': ('CSB 130', ' 1 pm'), 'CS430': ('CSB 425', ' 4 pm'), 'CS440': ('CSB 425', '11 am'), 'CS445': ('CSB 425', ' 3 pm'), 'CS453': ('CSB 325', '10 am'), 'CS464': ('CSB 130', ' 2 pm'), 'CS510': ('CSB 425', '10 am'), 'CS514': ('CSB 325', ' 3 pm'), 'CS535': ('CSB 130', ' 9 am'), 'CS540': ('CSB 425', '12 pm'), 'CS545': ('CSB 130', '11 am')}


In [6]:
display(assignments, rooms, times)

          CSB 130    CSB 325    CSB 425
----------------------------------------
 9 am      CS535      CS410      CS270
10 am      CS356      CS453      CS510
11 am      CS545      CS164      CS440
12 pm      CS414      CS220      CS540
 1 pm      CS420                 CS314
 2 pm      CS464      CS320      CS160
 3 pm      CS163      CS514      CS445
 4 pm      CS370      CS253      CS430


## Observation:

It took 2 steps to completely schedule the classes without violating any of the costraints.

## Grading

Download [A6grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A6grader.tar) and extract `A6grader.py` from it.  Grader will be available soon.

In [7]:
%run -i A6grader.py


======================= Code Execution =======================

Extracting python code from notebook named 'Kharat-A6.ipynb' and storing in notebookcode.py
Removing all statements that are not function or class defs or import statements.

Testing constraints_ok('CS410', ('CSB 130', ' 9 am'), 'CS510', ('CSB 130', ' 9 am'))

--- 10/10 points. Your constraints_ok function correctly returned False

Testing constraints_ok('CS410', ('CSB 130', ' 9 am'), 'CS510', ('CSB 130', '10 am'))

--- 10/10 points. Your constraints_ok function correctly returned True

Testing constraints_ok('CS410', ('CSB 130', '10 am'), 'CS430', ('CSB 425', '10 am')

--- 10/10 points. Your constraints_ok function correctly returned False

Testing  result, nsteps = schedule(classes, times, rooms, 100)

--- 30/30 points. Your schedule function returned a valid schedule.

Testing  call to schedule again with two more classes.

--- 30/30 points. Your schedule function correctly returned None.

C:\Users\NAMITA\Desktop\440 A

# Extra Credit

Solve the scheduling problem again but with the addition of
these preferences:

  * prefer schedules that do not schedule classes at 9 am, 12 pm or 4 pm.
  * prefer schedules with CS163 and CS164 meeting at 1 pm or 2 pm.

To accomplish this, do not modify `min_conflicts`.  Write another function that calls `min_conflicts` repeatedly, and for each solution found count the number of these new preferences that are not satisfied.  Remember the solution with the smallest count of unsatisfied preferences.  